In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(token="Your token")


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, DatasetDict
import pandas as pd
import datasets

In [ ]:
# **Step 2: Load Dataset (Only Input & Output)**
df = pd.read_csv("/content/Laws and Constitution of India1 - constitution_train_combined3.csv")
df_transformed = df.apply(lambda row: f"<s>[INST] {row['input']} [/INST] {row['output']} </s>", axis=1)

# Convert to Hugging Face dataset
dataset = datasets.Dataset.from_pandas(pd.DataFrame({"text": df_transformed}))

In [ ]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True
# )

# tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# # **Set padding token to avoid errors**
# tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     model_name, quantization_config=bnb_config, device_map="auto", use_auth_token=True
# )

# # **Step 3: Attach LoRA Adapters**
# lora_config = LoraConfig(
#     r=32,
#     lora_alpha=64,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()

# # **Step 4: Tokenize Data**
# def tokenize_function(example):
#     tokenized_inputs = tokenizer(
#         example["text"],
#         padding="max_length",
#         truncation=True,
#         max_length=512
#     )

#     # Labels should be the same as input_ids for causal LM training
#     tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()

#     return tokenized_inputs

# tokenized_datasets = dataset.map(tokenize_function, remove_columns=dataset.column_names, num_proc=1)

# # **Step 5: Split into Train (90%) & Test (10%)**
# split_dataset = tokenized_datasets.train_test_split(test_size=0.1)
# split_dataset = DatasetDict({"train": split_dataset["train"], "test": split_dataset["test"]})

# # **Step 6: Training Arguments (Optimized for LoRA + Low VRAM)**
# training_args = TrainingArguments(
#     output_dir="./fine_tuned_llama_7b",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     num_train_epochs=3,
#     save_total_limit=2,
#     logging_dir="./logs",
#     push_to_hub=False,
#     gradient_accumulation_steps=4,
#     logging_steps=10,
#     fp16=True,  # Enable mixed precision for efficiency
#     report_to="none",
#     optim="adamw_bnb_8bit",  # Optimized optimizer for 4-bit training
#     load_best_model_at_end=True
# )

# # **Step 7: Free Up GPU Memory Before Training**
# torch.cuda.empty_cache()

# # **Step 8: Define Trainer**
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=split_dataset["train"],
#     eval_dataset=split_dataset["test"],
#     tokenizer=tokenizer,
# )

# model.print_trainable_parameters()

# # **Step 9: Train the Model**
# trainer.train()

# # **Step 10: Save Model**
# model.save_pretrained("fine_tuned_llama_7b")
# tokenizer.save_pretrained("fine_tuned_llama_7b")

# print("Fine-tuning complete! Model saved in 'fine_tuned_llama_7b'.")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.2484


Map:   0%|          | 0/1796 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-2a6eb707c63f>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.2484


Epoch,Training Loss,Validation Loss
1,0.176300,0.214841
2,0.199200,0.202615
3,0.142300,0.201107


Fine-tuning complete! Model saved in 'fine_tuned_llama_7b'.


In [ ]:
pip install bitsandbytes
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# **Set padding token to avoid errors**
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map="auto", use_auth_token=True
)

# **Step 3: Attach LoRA Adapters**
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Apply LoRA to all key-value projections
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# **Step 4: Tokenize Data**
def tokenize_function(example):
    tokenized_inputs = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Labels should be the same as input_ids for causal LM training
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()

    # Ensure padding tokens do not contribute to loss
    tokenized_inputs["labels"] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokenized_inputs["labels"]
    ]

    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, remove_columns=dataset.column_names, num_proc=1)

# **Step 5: Split into Train (90%) & Test (10%)**
split_dataset = tokenized_datasets.train_test_split(test_size=0.1)
split_dataset = DatasetDict({"train": split_dataset["train"], "test": split_dataset["test"]})

# **Step 6: Training Arguments (Optimized for LoRA + Low VRAM)**
training_args = TrainingArguments(
    output_dir="./fine_tuned_llama_7b",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_total_limit=2,
    logging_dir="./logs",
    push_to_hub=False,
    gradient_accumulation_steps=4,
    logging_steps=10,
    fp16=True,  # Enable mixed precision for efficiency
    gradient_checkpointing=True,  # Reduce memory usage
    lr_scheduler_type="cosine",  # Learning rate decay
    learning_rate=2e-5,  # Lower LR for stable fine-tuning
    optim="adamw_bnb_8bit",  # Optimized optimizer for 4-bit training
    warmup_steps=50,  # Warmup for better stability
    save_safetensors=True,  # Save model in a memory-efficient format
    load_best_model_at_end=True
)

# **Step 7: Free Up GPU Memory Before Training**
torch.cuda.empty_cache()

# **Step 8: Define Trainer**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
)

model.print_trainable_parameters()

# **Step 9: Train the Model**
trainer.train()

# **Step 10: Save Model**
model.save_pretrained("fine_tuned_llama_7b")
tokenizer.save_pretrained("fine_tuned_llama_7b")

print("Fine-tuning complete! Model saved in 'fine_tuned_llama_7b'.")

SyntaxError: invalid syntax (<ipython-input-9-6936910e2b98>, line 1)

In [ ]:
from transformers import pipeline

model_path = "fine_tuned_llama_7b"
generator = pipeline("text-generation", model=model_path, tokenizer=model_path)

prompt = "What are the fundamental rights in the Indian Constitution?"
result = generator(prompt, max_length=200, do_sample=True)
print(result[0]["generated_text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pip install  bitsandbytes

In [ ]:
print("Tokenizer:", tokenizer)
print("Model:", model)


In [1]:
pip freeze 'requirements.txt'

absl-py==1.4.0
accelerate==1.3.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.13
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.3.10.0.29.26
astunparse==1.6.3
atpublic==4.1.0
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.40.0
bigquery-magics==0.8.0
bleach==6.2.0
blinker==1.9.0
blis==1.2.0
blosc2==3.2.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.